In [1]:
from PIL import Image
from src.circle_detection import detect_circles, hough_circle_transform, draw_detected_circles, np2img
from src.edge_detection import edge_detection, compress_image, apply_gaussian_blur, compute_gradients, non_maximum_suppression, double_threshold
import os

def list_image_names(directory):
    """
    列出指定目录中所有图片文件的名称，不包括子目录。
    """
    images = []
    supported_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
    for file in os.listdir(directory):
        if file.endswith(supported_extensions):
            images.append(file)
    return images


directory_path = './data/test_images'
results_path = './data/results'
images = list_image_names(directory_path)

for image_name in images:
    if not os.path.exists(f'{results_path}/{image_name}'):
        os.makedirs(f'{results_path}/{image_name}')
    
    image = Image.open(f'./data/test_images/{image_name}').convert('RGB')
    image = compress_image(image, base_width=400)  # 调整图像到固定大小，提高后续参数的可用性
    blurred_image = apply_gaussian_blur(image)  # 参数可调
    blurred_image.save(f'{results_path}/{image_name}/blurred_image.png')

    gradient_magnitude, gradient_direction = compute_gradients(blurred_image)
    np2img(gradient_magnitude).save(f'{results_path}/{image_name}/gradient.png')

    nms_image = non_maximum_suppression(gradient_magnitude, gradient_direction)
    edges = double_threshold(nms_image, low_threshold=70 , high_threshold=90)  # 参数可调
    np2img(edges).save(f'{results_path}/{image_name}/edges.png')
    
    circles = hough_circle_transform(edges, radius_range=(15,100))  # 参数可调
    result = draw_detected_circles(image, circles)
    result.save(f'{results_path}/{image_name}/result.png')

Detected circles: [(160, 205, 20), (118, 276, 22), (350, 129, 20), (47, 270, 29), (221, 201, 25), (45, 205, 20), (105, 203, 23), (349, 45, 30), (362, 194, 33), (283, 123, 25), (254, 274, 25), (106, 130, 20), (46, 129, 21), (162, 53, 24), (212, 129, 20), (289, 54, 20), (311, 274, 25), (225, 53, 23), (373, 277, 23), (283, 200, 22), (104, 60, 21), (162, 132, 18), (183, 271, 26)]
Detected circles: [(323, 175, 45), (296, 94, 72), (70, 168, 49), (205, 137, 61), (123, 130, 45), (283, 133, 15)]


以下为easy version，可供参考

In [2]:
import cv2
from PIL import Image
import numpy as np
import os

def resize_image_by_width(image, new_width):
    """
    按比例调整图像宽度。

    参数:
    - image: 原始图像
    - new_width: 新的宽度

    返回:
    - resized_image: 调整大小后的图像
    """
    # 获取原始图像的尺寸
    (h, w) = image.shape[:2]

    # 计算宽度的缩放比例，并据此计算新的高度
    ratio = new_width / float(w)
    new_height = int(h * ratio)

    # 调整图像大小
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    return resized_image

def list_image_names(directory):
    """
    列出指定目录中所有图片文件的名称，不包括子目录。
    """
    images = []
    supported_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
    for file in os.listdir(directory):
        if file.endswith(supported_extensions):
            images.append(file)
    return images


def detect_coins(image_path, save_path):
    # 读取图像
    img = cv2.imread(image_path)

    resized_image = resize_image_by_width(img, 400)

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    
    # # 高斯模糊
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # Canny边缘检测
    edges = cv2.Canny(blurred, threshold1=70, threshold2=90)
    # 显示边缘检测的结果
    cv2.imwrite(save_path, edges)

directory_path = './data/test_images'
results_path = './data/results'
images = list_image_names(directory_path)
for image_name in images:
    save_path = f'{results_path}/{image_name}/cv2_edges.png'
    detect_coins(f'./data/test_images/{image_name}', save_path)

In [7]:
for image_name in images:
    image = Image.open(f'./data/test_images/{image_name}').convert('RGB')
    image = compress_image(image, base_width=400)  # 调整图像到固定大小，提高后续参数的可用性
    blurred_image = apply_gaussian_blur(image, kernel_size=45, sigma=2)  # 参数可调
    gradient_magnitude, gradient_direction = compute_gradients(blurred_image)
    nms_image = non_maximum_suppression(gradient_magnitude, gradient_direction)
    edges = double_threshold(nms_image, low_threshold=70 , high_threshold=90)  # 参数可调
    np2img(edges).show()
    break